<img src='../../img/anaconda-logo.png' align='left' style="padding:10px">
<br>
*Copyright Continuum 2012-2016 All Rights Reserved.*

# Datashader GIS Example: Taxi rides

## Table of Contents
* [Datashader GIS Example: Taxi rides](#Datashader-GIS-Example:-Taxi-rides)
	* [Set-Up](#Set-Up)
* [The Data Set](#The-Data-Set)
* [Preparation](#Preparation)
* [The Pipeline](#The-Pipeline)
	* [Aggregation and Transform](#Aggregation-and-Transform)
	* [Colormapping Transformations](#Colormapping-Transformations)
	* [Aggregation by Average Distance](#Aggregation-by-Average-Distance)
* [Interactive Visualizations](#Interactive-Visualizations)
	* [Use a Better Basemap](#Use-a-Better-Basemap)
* [Exercise](#Exercise)


## Set-Up

In [ ]:
import pandas as pd
import numpy as np

# The Data Set

The data set used here is NYC taxi rides from April 1, 2015.

In [ ]:
taxi = pd.read_csv('data/April1.csv')
taxi.head()

In [ ]:
taxi.info()

# Preparation

Latitude and longitude positions must be transformed to Meters from the prime meridian (*mercator*) units for mapping overlays.

In [ ]:
from pyproj import Proj, transform
inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:3857') 

# "epsg:4326" is a web code for a coordinate reference frame
# the coordinate space "epsg:4326" (WGS84) is in lat/lon
#    http://spatialreference.org/ref/epsg/wgs-84/
# the coordinate space "epsg:3857" (Web Mercator) is measure in meters
# WebMercator says the Earth is a square, and aquares are easy to tile


# Now let's apply the projection from one coordinate space for another
taxi['pickup_x'], taxi['pickup_y'] = \
  transform(inProj, outProj, taxi['pickup_longitude'].values, taxi['pickup_latitude'].values)

In [ ]:
# Alternative, if you have successfully run `master_download.py` and have the full data set for NYC taxis
# taxi = pd.read_csv('../../data/Datashader/nyc_taxi.csv')

In [ ]:
import datashader as ds
from datashader import transfer_functions as tf
from datashader.colors import Greys9
Greys9_r = list(reversed(Greys9))[:-2]

# The Pipeline

## Aggregation and Transform

Aggregate the data by number of rides.

In [ ]:
x_range=(-8250000,-8210000)
y_range=(4965000,4990000)

cvs = ds.Canvas(plot_width=800, plot_height=500, x_range=x_range, y_range=y_range)
agg = cvs.points(taxi, 'pickup_x', 'pickup_y')
tf.shade(agg, cmap=Greys9_r, how='eq_hist')


## Colormapping Transformations

Color mapping: blue indicates higher density of rides.

In [ ]:
from datashader.colors import inferno

x_range=(-8250000,-8210000)
y_range=(4965000,4990000)

cvs = ds.Canvas(plot_width=800, plot_height=500, x_range=x_range, y_range=y_range)
agg = cvs.points(taxi, 'pickup_x', 'pickup_y')
tf.shade(agg, cmap=reversed(inferno), how='eq_hist')

## Aggregation by Average Distance

Aggregate by average distance.

Rides from the airport are longer than most others.

Once you're in the city why leave?

In [ ]:
x_range=(-8250000,-8210000)
y_range=(4965000,4990000)

cvs = ds.Canvas(plot_width=800, plot_height=500, x_range=x_range, y_range=y_range)
agg = cvs.points(taxi, 'pickup_x', 'pickup_y', ds.mean('trip_distance'))
tf.shade(agg, cmap=reversed(inferno), how='eq_hist')

# Interactive Visualizations

<div class='alert alert-info'>
<img src='img/topics/Advanced-Concept.png' align='left' style='padding:10x'>
<br>
<big><big>
Combine Datashader and Bokeh for interative Big Data visualizations.
</big></big>
<br><br>
</div>

The new tool used here will be `Dynspread`, which allows the points to expand as plot is zoomed.

In [ ]:
def make_img(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.points(taxi, 'pickup_x', 'pickup_y')
    img = tf.shade(agg, cmap=inferno, how='eq_hist')
    return tf.dynspread(img, threshold=0.5, max_px=4)

In [ ]:
from bokeh.io import output_notebook, show
output_notebook()

Watch how the the plot changes as you zoom in or out.

Looking at the `make_img` function above why do you think the colors change?

In [ ]:
import bokeh
bokeh.__version__

In [ ]:
!conda install bokeh -y

In [ ]:
from bokeh.plotting import figure
from bokeh.tile_providers import STAMEN_TERRAIN
from datashader.bokeh_ext import InteractiveImage

ranges = {
    'x_range':(-8236013,-8234013),
    'y_range':(4971883,4981883)
}

p = figure(background_fill_color="black",responsive=True, plot_width=900,
          **ranges)
p.axis.visible = False
p.grid.visible = False   # bokeh 0.12.1
#p.grid.grid_line_alpha = 0 # bokeh 0.12.0


p.add_tile(STAMEN_TERRAIN)

InteractiveImage(p, make_img)

## Use a Better Basemap

> *"Your basemap should provide context, but not distract nor hide your data." -- Master Collins*

In [ ]:
def base_plot(tools='pan,wheel_zoom,reset',plot_width=900, plot_height=600, x_range=None, y_range=None, **plot_args):
    p = Figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
        x_range=x_range, y_range=y_range, outline_line_color=None,
        min_border=0, min_border_left=0, min_border_right=0,
        min_border_top=0, min_border_bottom=0, **plot_args)
    
    p.axis.visible = False
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.add_tile(STAMEN_TONER, alpha=.5)
    return p

In [ ]:
from bokeh.plotting import figure
from bokeh.tile_providers import STAMEN_TONER ####### chnged this too ######
from datashader.bokeh_ext import InteractiveImage

ranges = {
    'x_range':(-8236013,-8234013),
    'y_range':(4971883,4981883)
}

p = figure(background_fill_color="black",responsive=True, plot_width=900,
          **ranges)
p.axis.visible = False
p.grid.visible = False   # bokeh 0.12.1
#p.grid.grid_line_alpha = 0 # bokeh 0.12.0


p.add_tile(STAMEN_TONER, alpha=.5)  ####### Changed this line ######

InteractiveImage(p, make_img)

# Exercise

<img src='img/topics/Exercise.png' align='left' style='padding:10px'>
<br>
<a href='./Datashader_ex_taxi.ipynb' class='btn btn-primary btn-lg'>Taxi Data</a>